In [21]:
from keras.layers import *
from keras.models import Model
from keras import optimizers
import numpy as np
from tensorflow_model_optimization.quantization.keras import vitis_quantize
import os

def building_block(X, filter_size, filters, stride=1):

    # Save the input value for shortcut
    X_shortcut = X

    # Reshape shortcut for later adding if dimensions change
    if stride > 1:

        X_shortcut = Conv2D(filters, (1, 1), strides=stride, padding='same')(X_shortcut)
        X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # First layer of the block
    X = Conv2D(filters, kernel_size = filter_size, strides=stride, padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second layer of the block
    X = Conv2D(filters, kernel_size = filter_size, strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = add([X, X_shortcut])  # Add shortcut value to main path
    X = Activation('relu')(X)

    return X


# Full model
def create_model(input_shape, classes, name):

    # Define the input
    X_input = Input(input_shape)

    # Stage 1
    X = Conv2D(filters=16, kernel_size=3, strides=(1, 1), padding='same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Stage 2
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)

    # Stage 3
    X = building_block(X, filter_size=3, filters=32, stride=2)  # dimensions change (stride=2)
    X = building_block(X, filter_size=3, filters=32, stride=1)
    X = building_block(X, filter_size=3, filters=32, stride=1)
    X = building_block(X, filter_size=3, filters=32, stride=1)
    X = building_block(X, filter_size=3, filters=32, stride=1)

    # Stage 4
    X = building_block(X, filter_size=3, filters=64, stride=2)  # dimensions change (stride=2)
    X = building_block(X, filter_size=3, filters=64, stride=1)
    X = building_block(X, filter_size=3, filters=64, stride=1)
    X = building_block(X, filter_size=3, filters=64, stride=1)
    X = building_block(X, filter_size=3, filters=64, stride=1)

    # Average pooling and output layer
    X = AveragePooling2D()(X)
    X = AveragePooling2D()(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name=name)

    return model


shape_range =range(192,512,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    input_shape = (shape_x,shape_y,1)
    model = create_model(input_shape=input_shape, classes=10, name='ResNet32')
    #model.summary()
    
    #model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #history = model.fit(X_train, Y_train, epochs=1)
    print(model.get_weights()[0])
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:10], weight_bit=8, activation_bit=8)
    quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('quantized.h5')
    !vai_c_tensorflow2 \
    --model ./quantized.h5 \
    --arch ./arch.json \
    --output_dir ./xmodel_resnet32 \
    --net_name deploy
    os.rename("./xmodel_resnet32/deploy.xmodel","./xmodel_resnet32/resnet32_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))


[[[[ 1.65870920e-01 -1.67279035e-01 -1.81791037e-02 -1.79208368e-02
    -1.17555276e-01 -1.33732274e-01  7.45451599e-02  4.15281802e-02
    -8.48461837e-02  1.18924871e-01 -3.51667702e-02  3.87775004e-02
     8.86385888e-02 -3.68226469e-02  1.96354881e-01 -2.63899565e-05]]

  [[ 1.46789327e-01 -1.23896815e-01 -7.98075646e-02  1.51929125e-01
    -7.02607632e-02  3.56483459e-02 -6.22659922e-04 -1.56545371e-01
     1.81671098e-01  1.88558295e-01 -1.25591993e-01  2.17340738e-02
     1.03183642e-01 -3.26841623e-02 -1.34394348e-01  1.26916602e-01]]

  [[-1.49249703e-01  1.39825419e-01  1.16058454e-01 -1.61670774e-01
    -4.25378978e-02 -3.10295969e-02  1.71304479e-01  6.16714358e-04
    -5.80496043e-02  1.23701617e-01  4.56832349e-03 -1.47348940e-02
     2.76243687e-03  7.97053128e-02 -1.60587549e-01 -9.19972509e-02]]]


 [[[-3.75774056e-02  1.26025230e-02 -1.33077562e-01 -1.39812231e-02
     1.89975336e-01  1.68654248e-01 -1.74085677e-01 -3.29454839e-02
     1.05694905e-01 -1.89528018e-02 -

KeyboardInterrupt: 